In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools

In [ ]:
data1, data2, data3 = pd.read_json('./data1.json'), pd.read_json('./data2.json'), pd.read_json('./data3.json')

frames = [data1, data2, data3]

data = pd.concat(frames, sort=False)
trainSet, testSet = train_test_split(data, test_size=0.1)
estimators = [x for x in range(300, 10001, 100)]
lambdas = [10**-1, 10**-2, 10**-3, 10**-4]
depths = [x for x in range(15, 51, 5)]
mix = [estimators,lambdas,depths]
textFile = 'xgtest.txt'
#itertools.combinations(mix, 3)
for i, comb in enumerate(itertools.product(*mix)):
    with open(textFile, 'a') as f:
            f.write(f'Set #:{i} with parameters: {comb} \n')

    regressor = xgb.XGBRegressor(
        n_estimators = comb[0],
        reg_lambda=comb[1],
        gamma = 0,
        max_depth=comb[2]
    )
    regressor.fit(trainSet.iloc[:,:16], trainSet.iloc[:,16:])

    output_pred = regressor.predict(testSet.iloc[:,:16])
    offset = 0
    total = testSet.shape[0]
    for i in range(len(output_pred)):
        offset += (output_pred[i] * 100) / testSet.iloc[i,16:].output
        #print(f'Prediction: {output_pred[i]}. Value: {testSet.iloc[i,16:].output}')
    with open(textFile, 'a') as f:
            f.write(f'The test were off by an average of: {offset/total:.2f}% \n')

In [6]:
mean_squared_error(testSet.iloc[:,16:], output_pred)

11010.39015724753